# BE/Bi 103, Fall 2016: Homework 7
## Due 1pm, Sunday, November 13

(c) 2016 Justin Bois and Heidi Klumpe. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained therein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

*This homework was generated from an Jupyter notebook.  You can download the notebook [here](hw7.ipynb). You can also view it [here](https://nbviewer.jupyter.org/url/bebi103.caltech.edu/2016/homework/hw7.ipynb).*

In [28]:
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats as st
import numba

import bebi103

import matplotlib.pyplot as plt
import seaborn as sns
rc={'lines.linewidth': 2, 'axes.labelsize': 14, 'axes.titlesize': 14}
sns.set(rc=rc)
%matplotlib inline

### Problem 7.1: Practice writing posteriors (25 points + 10 points extra credit)

This problem is a worth a total of 25 points. You can do any subset of these problems to get full (or full plus extra) credit.

**a)** (10 pts) Write and/or draw a flow chart the details the steps from "Write Bayes' Theorem" to arriving at the final form of the posterior for a parameter estimation problem. If you hand-sketch the flow chart, you can include it in your Jupyter notebook as a scan. To include an image with Markdown, so this:

    ![description of image](file_of_image.png)
    
Be sure to include the image itself in your repository.

**b)** The Elowitz lab is interested in the design principles of cellular signaling pathway architectures, or how the interactions between signaling pathway components (things like extracellular ligands and receptors) give rise to different signal processing capabilities. Below are some experiments we might run to get a better quantitative understanding of cell signaling. 

For each of the following scenarios:
* (3 pts) Write the full form of the likelihood and prior you would use to estimate the parameter(s). **You must define all symbols (e.g. parameters and variables).**
* (2 pts) Explain why you chose the form you did, including what you chose to neglect or exclude. More than one version may be appropriate, so give convincing reasons to select the form you wrote.

**Exercise 1**
You have images of many fields of cells, where fluorescently-labeled receptors at the cell surface appear as dots. (Assume these are maximum projections of confocal images, so that the image includes the entire cell membrane). You would like to estimate the mean number of receptors expressed by this cell line, and are using an automated image analysis tool that can count the number of dots on each cell. A previous paper reported that there are are $10^6 \pm 10^3$ of this receptor type expressed by this cell line.

**Exercise 2**
You decide to get higher throughput counts by flowing your fluorescently-labeled cells. However, first you need to know how the fluorescence depends on the number of fluorophores, which we presume to be equal to the number of receptors. To start approximating this, you measure the fluorescence of beads ($F$) with a known number of fluorophores attached to each ($N$). You assume the fluorescence depends linearly on the number of fluorophores, and that there is some background fluorescence: 

\begin{align}
F(N \mid a, b) = aN + b.
\end{align}

We are interested in estimating the values of $a$ and $b$ (though we recognize the background fluorescence will probably be different in our cells).

**Exercise 3**
A fully-formed signaling complex requires two receptor subunits, and sometimes these receptors come together spontaneously. You want to estimate the average rate at which this happens. You put one half of a fluorescent protein on one receptor subunit and the other half on the other receptor subunit. If the subunits spontaneously come together, you will observe a fluorescent dot. You use time-lapse microscopy to image these cells, and generate a list of the times between dots on given single cells (e.g. you waited $t_1$, $t_2$, etc. seconds between seeing receptors spontaneously come together on cell $j$).

**Exercise 4**
You weren't able to do the experiment described in **Exercise 3**, but you read about it in a paper and request the data. They send you a table that reports the number of times receptor complexes spontaneously formed in ten minute intervals for each cell they analyzed. As above, you still want to estimate the average rate at which this happens. You also have an estimate and error bar for this rate from a different paper.

**Exercise 5**
You are studying a signaling pathway that, when the ligand binds the receptor, forms a multimeric protein complex at the intracellular side of the membrane. You assume that the rate of each protein joining this complex is roughly the same. You want to estimate the number of proteins in this complex, and the average rate at which they join. You employ a similar approach as in **Exercise 3**, using protein fusions that produce a fluorescent signal only when the complex is fully-formed. You use time-lapse microscopy to time how long it takes the complex to fully form after you add the ligand.

### _Problem 7.1 Solution_

### Part A (10 points) 

Write and/or draw a flow chart the details the steps from "Write Bayes' Theorem" to arriving at the final form of the posterior. Be as detailed as possible.

_Sample Solution:_

![Part A Solution](prob7.1a_solution.svg)

### Part B (5 points ea.)

The Elowitz lab is interested in the design principles of signaling pathway architecture, or how the interactions between signaling pathway components (things like extracellular ligands and receptors) give rise to different signal processing capabilities. Below are some experiments we might run to get a better quantitative understanding of cell signaling. 

For each of the following scenarios:
* (3 pts) Write the full form of the posterior you would use to estimate the parameter(s). **You must define all symbols (e.g. parameters and variables) for full credit.**
* (2 pts) Explain why you chose that form of the posterior, including what you chose to neglect or exclude. More than one version of the posterior may be appropriate, so give convincing reasons to select the form you wrote.

_Note on these solutions:_

For all my solutions, I neglect the evidence. As it's not a function of the parameter values, we can find the parameter values that maximize the posterior without knowing its value. Also, all my solutions will have the following format:

\begin{align}
\\ \mathrm{posterior} \propto \left( \mathrm{likelihood} \right) \left( \mathrm{prior} \right) 
\end{align}

Note too that none of priors are properly normalized, and there are no bounds on the prior (i.e. I don't specify that it should be zero outside the allowed parameter range). If you're unsure how that changes the functional form of the posterior, you can see the flow chart I drew for **Part A** to see how that would be done.

#### Exercise 1: 
You have images of many fields of cells, where fluorescently-labeled receptors at the cell surface appear as dots. (Assume these are maximum projections of confocal images, so that the image includes the entire cell membrane). You would like to estimate the mean number of receptors expressed by this cell line, and are using an automated image analysis tool that can count the number of dots on each cell. A previous paper reported that there are are $10^6 \pm 10^3$ of this receptor type expressed by this cell line.

_Functional form_: 

\begin{align}
\\ P(\mu_R, \sigma_R \mid \textbf{N}, \mu_{pub}, \sigma_{pub}) \propto \left( \prod_{i \in \textbf{N}} \frac{1}{N_i \sqrt{2\pi\sigma_R^2}}\, \mathrm{e}^{-\left(\ln N_i - \ln \mu_R\right)^2 / 2\sigma_R^2} \right) \left( \frac{1}{\sigma_R} \frac{1}{\mu_R \sqrt{2 \pi \sigma_{pub}^2}}\, \mathrm{e}^{-(\ln \mu_R - \ln \mu_{pub})^2 / 2 \sigma_{pub}^2} \right)
\end{align}

where:  

$\textbf{N}$ is the array of measurements (number of receptors per cell),  
$N_i$ is an individual measurement,  
$i$ is the index over the number of measurements,  
$\mu_R$ is the mean of the receptor number,  
$\sigma_R^2$ is the variance of the receptor number,  
$\mu_{pub}$ is the published mean of the receptor number, and  
$\sigma_{pub}^2$ is the published variance of the receptor number.

_Explanation_: Based on the published estimate, receptor number appears to be logarithmically distributed. We may  decide that the number of receptors is probably broadly distributed, and so use a [**log-Cauchy**](https://en.wikipedia.org/wiki/Log-Cauchy_distribution) distribution for the likelihood, but here I  just right a log-normal distribution. This describes the statistical error, that is, how I expect the measurement ($\textbf{N}$) to vary from some mean value of receptor expression ($\mu_R$). Also, because we have an estimate for the receptor number, I use an informative prior that is also log-normal. I use an uninformative, Jeffreys prior for the error in the measurement ($\sigma_R$), as I'm unsure how it should vary. 

#### Exercise 2:
You decide to get higher throughput counts by flowing your fluorescently-labeled cells. However, first you need to know how the fluorescence depends on the number of fluorophores, which we presume to be equal to the number of receptors. To start approximating this, you measure the fluorescence of beads ($F$) with a known number of fluorophores attached ($N$). You assume the fluorescence depends linearly on the number of fluorophores, and that there is some background fluorescence: 

\begin{align}
\\ F(N \mid a, b) = aN + b
\end{align}

We are interested in estimating the values of $a$ and $b$, though we recognize the background fluorescence will probably be different in our cells.

_Functional form_: 

\begin{align}
\\ P(a, b, \sigma_F \mid \textbf{F}, \textbf{N}, I) \propto \left( \prod_{i \in \textbf{F}} \frac{1}{\sqrt{2\pi\sigma_F^2}}  \mathrm{exp} \left( -\frac{(F_i - (aN_i + b))^2}{2 \sigma_F^2} \right) \right)\left( \frac{1}{\sigma_F}  \frac{1}{a} \right)
\end{align}

where:  

$\textbf{F}$ is the array of measurements (fluorescence of each bead),  
$F_i$ is an individual fluorescence measurement,  
$N_i$ is the number of receptors associated with fluorescence measurement $F_i$,  
$i$ is the index over the number of measurements,  
$a$ and $b$ are the parameters from our math model for $F$, and  
$\sigma_F^2$ is the variance of the fluorescence measurement.  

_Explanation_: We assume that our fluorescent measurements are actually quite good, such that they are normally distributed about our expectation from the math model. Thus, we choose a Gaussian distribution for our likelihood (note that this introduces an extra parameter, $\sigma_F$, which describes how the measurement varies from the math model prediction). We also take uninformative priors for all parameters. Scale parameters $\sigma_F$ and $a$ use the familiar Jeffreys prior. Location parameter $b$ gets a uniform prior (i.e. is 1 everywhere in the allowed parameter range), so does not appear. 

Note that this actually an incorrect implementation of the Jeffreys prior. Per Justin's [**prior tutorial**](http://bebi103.caltech.edu/2016/tutorials/aux3_priors.html), for a linear regression, a true Jeffreys prior would be:

\begin{align}
\\ P(a, b \mid I) = \frac{1}{2 (1 + a^2)^{\frac{3}{2}}}
\end{align}

However, given sufficient data points, the slight inaccuracy in the form of the prior should not affect our estimate that much.

#### Exercise 3:
A fully-formed signaling complex requires two receptor subunits, and sometimes these receptor come together spontaneously. You want to estimate the average rate at which this happens. You put one half of a fluorescent protein on one receptor subunit and the other half on the other receptor subunit. If the subunits spontaneously come together, you will observe a fluorescent dot. You use time-lapse microscopy to image these cells, and generate a list of the times between dots on single cells (e.g. you waited $t_1$, $t_2$, etc. seconds between seeing receptors spontaneously come together).

_Functional form_: 

\begin{align}
\\ P(\lambda \mid \textbf{t}, I) \propto \left( \prod_{i \in \textbf{t}} \frac{1}{\lambda}  \mathrm{exp} \left( -\frac{t_i}{\lambda} \right) \right)\left( 1 \right)
\end{align}

where:  

$\textbf{t}$ is the array of measurements (times for each spontaneous receptor assembly),  
$t_i$ is an individual fluorescence measurement,  
$i$ is the index over the number of measurements, and  
$\lambda$ is the average rate of spontaneous receptor assembly.  

_Explanation_: We're trying to estimate the rate ($\lambda$) of a relatively rare event, the spontaneous assembly of two receptor subunits. The waiting times ($\textbf{t}$) for such Poisson events are exponentially distributed. We take a uniform prior for the rate, as we have no information about it. 

#### Exercise 4
You weren't able to do the experiment described in **Exercise 3**, but you read about it in a paper and request the data. They send you a table that reports the number of times receptor complexes spontaneously formed in ten minute intervals. As above, you still want to estimate the average rate at which this happens. You also have an estimate and error bar for this rate from a different paper.

_Functional form_: 

\begin{align}
\\ P(\lambda, \sigma_{\lambda} \mid \textbf{N}, \lambda_{pub}) \propto \left( \prod_{i \in \textbf{N}} \frac{\lambda^{N_i}}{N_i!}  \mathrm{exp} \left( -\lambda \right) \right) \left( \frac{1} {\sqrt{2 \pi \sigma_{\lambda}^2}} \mathrm{exp} \left( - \frac{(\lambda - \lambda_{pub})^2}{2\sigma_{\lambda}^2} \right) \right)
\end{align}

where:  

$\textbf{N}$ is the array of measurements (number of spontaneous receptor assemblies in ten minutes),  
$N_i$ is an individual count of receptor assemblies in ten minutes,  
$i$ is the index over the number of measurements,  
$\lambda$ is the average rate of spontaneous receptor assembly per ten minutes,  
$\lambda_{pub}$ is the published estimate of that rate, and  
$\sigma_{\lambda}$ is the published error in $\lambda$.

_Explanation_: We're trying to estimate the rate ($\lambda$) of a relatively rare event, the spontaneous assembly of two receptor subunits. The number of events we observe in a given time interval ($\textbf{N}$) is Poisson distributed. Because of the published rate $\lambda_{pub}$, we have an informative prior for $\lambda$.

#### Exercise 5:
You are studying a signaling pathway that, when the ligand binds the receptor, forms a multimeric protein complex at the intracellular side of the membrane. You assume that the rate of each protein joining this complex is roughly the same. You want to estimate the number of proteins in this complex, and the average rate at which they join. You employ a similar approach as in **Exercise 3**, using protein fusions that produce a fluorescent signal only when the complex is fully-formed. You use time-lapse microscopy to time how long it takes the complex to fully form after you add the ligand.

_Functional form_: 

\begin{align}
\\ P(a, r \mid \textbf{t}, I) \propto \left( \prod_{i \in \textbf{t}} \frac{1}{\Gamma \left(a\right)} \frac{(rt_i)^a}{t_i}  \mathrm{exp} \left( -rt_i \right) \right) \left( 1 \right)
\end{align}

where:  

$\textbf{t}$ is the array of measurements (waiting times for full complex assembly),  
$t_i$ is one waiting time,  
$i$ is the index over the number of measurements,  
$r$ is the average rate of each protein binding the complex, and  
$a$ is the number of binding events (or one less than the number of proteins in the complex).

_Explanation_: We're trying to estimate the rate ($r$) of each protein joining the receptor complex, as well as how many proteins join the complex ($a$). This story matches the Gamma distribution, which is the amount of time we have to wait for $a$ arrivals of a Poisson process, given that the average arrival time is $r$. Also, we don't have any information about $a$ or $r$, and the functional form of the likelihood makes it difficult to determine if we should treat them as location or scale parameters (thought I suspect $r$ is more like a scale or shape parameter, and $a$ is a location parameter). So, I just take a sufficiently flat prior (i.e. 1 everywhere within a reasonable range for these parameter values).


<br />

### Problem 7.2: Hacker stats and Darwin's finches (75 pts + 25 pts extra credit)

Peter and Rosemary Grant of Princeton University have visited the island of Daphne Major on the Galápagos every year for over forty years and have been taking a careful inventory of the finches there. The Grants recently published a wonderful book,  [40 years of evolution: Darwin's finches on Daphne Major Island](http://www.worldcat.org/oclc/854285415). They were generous and made their data publicly available on the [Dryad data repository](http://dx.doi.org/10.5061/dryad.g6g3h). (In general, it is a very good idea to put your published data in public data repositories, both to preserve the data and also to make your findings public.) We will be using this data set to learn about evolution of Darwin's finches and use your hacker statistics skills. Up until part (f), all of your analyses will use nonparametric frequentist hacker stats.

We will focus on the primary two species of ground finch on Daphne Major, *Geospiza fortis* and *Geospiza scandens*. In this [data set](../data/finch_beaks.csv), you will find measurements of the beak length (tip to base) and beak depth (top to bottom) of these finches in the years 1973, 1975, 1987, 1991, and 2012. Also included in that data set is the band number for the bird, which gives a unique identifier.

**a)** We start with a little tidying of the data. Think about how you will deal with duplicate measurements of the same bird and make a decision on how those data are to be treated.

**b)** Plot ECDFs of the beak depths of *Geospiza scandens* in 1975 and in 2012. Then, estimate the mean beak depth in for each of these years with confidence intervals.

**c)** Perform a hypothesis test comparing the *G. scandens* beak depths in 1975 and 2012. Carefully state your null hypothesis, your test statistic, and you definition of what it means to be at least as extreme as the test statistic. Comment on the results. It might be interesting to know that a severe drought in 1976 and 1977  resulted in the death of the plants that produce small seeds on the island.

**d)** Devise a measure for the *shape* of a beak. That is, some scalar measure that combines both the length and depth of the beak. Compare this measure between species and through time. (This is very open-ended. It is up to you to define the measure, make relevant plots, compute confidence intervals, and possibly do hypothesis tests to see how shape changes over time and between the two species.)

**e)** Introgressive hybridization, occurs when a *G. scandens* bird mates with a *G. fortis* bird, and then the offspring mates again with pure *G. scandens*. This brings traits from *G. fortis* into the *F. scandens* genome. As this may be a mode by which beak geometries of *G. scandens* change over time, it is useful to know how *heritable* a trait is. Heritability is defined as the ratio of the covariance between parents and offsprings to the *variance of the parents alone*. To be clear, the heritability is defined as follows.

1. Compute the average value of a trait in a pair of parents.
2. Compute the average value of that trait among the offspring of those parents.
3. Do this for each set of parents/offspring. Using this data set, compute the covariance among all average offspring and the variance among all average parents.

This is a more apt definition than, say, the Pearson correlation, because it is a direct comparison between parents and offspring. 

Heritability data for beak depth for *G. fortis* and *G. scandens* can be found [here](../data/fortis_beak_depth_heredity.csv) and [here](../data/scandens_beak_depth_heredity.csv), respectively. (Be sure to look at the files before reading them in; they do have different formats.) From these data, compute the heritability of beak depth in the two species, with confidence intervals. How do they differ, and what consequences might this have for introgressive hybridization?

**f)** (25 pts extra credit) Repeat all of the above analysis using parametric Bayesian modeling.